<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0120EN-SkillsNetwork/images/IDSN-logo.png" width="300" alt="cognitiveclass.ai logo">

# Eager Execution in TensorFlow 2.X

Estimated time needed: **15** minutes

## Objectives

After completing this lab you will be able to:

- Understand the impact of eager execution and the need to enable it


<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3><strong>In this notebook we will overview Eager Execution in  TensorFlow 2.x</strong></font>
<br>
<h2>Table of Contents</h2>
<ol>
    <li>Instructions</li>
    <li>Eager Execution</li>
    <li>Tensorflow Operations Without Eager Execution Mode</li>
    <li>Tensorflow Operations With Eager Execution Mode</li>
    <li>Dynamic Control Flow</li>
    <li>Thank You</li>
</ol>
<p></p>
</div>
<br>

<hr>


# Instructions


## Installing TensorFlow


We begin by installing TensorFlow version 2.9.0 and its required prerequistes.


In [ ]:
!pip install grpcio==1.24.3
!pip install tensorflow==2.9.0

#### <b>Notice:</b> This notebook has been created with TensorFlow version 2.9.0, and might not work with other versions. Therefore we check:


In [1]:
import tensorflow as tf
if not tf.__version__ == '2.9.0':
    print(tf.__version__)
    raise ValueError('please upgrade to TensorFlow 2.9.0, or restart your Kernel (Kernel->Restart & Clear Output)')

# Eager Execution 


TensorFlow's **eager execution** is an imperative programming environment that evaluates operations immediately, without building graphs, operations return concrete values instead of constructing a computational graph to run later. This makes it easy to get started with TensorFlow and debug models.


With **TensorFlow 2.x**, **Eager Execution is enabled by default**. This allows TensorFlow code to be executed and evaluated line by line. Before version 2.x was released, every graph had to be run wihthin a TensorFlow **session**. This only allowed for the entire graph to be run all at once. This made it hard to debug the computation graph. 


Eager execution is a flexible machine learning platform for research and experimentation, providing:

- **An intuitive interface**-Structure your code naturally and use Python data structures. Quickly iterate on small models and   small data.


- **Easier debugging**- Execute operations directly to inspect code line by line and test changes. Use standard Python debugging tools for immediate error reporting.


- **Natural control flow**—Use Python control flow instead of graph control flow, simplifying the specification of dynamic models.


As I mentioned above, in **Tensorflow 2.x**, eager execution is enabled by default. To verify that please run the below code.


In [2]:
tf.executing_eagerly()

True

Now you can run TensorFlow operations and the results will return immediately:


But first let me show you how things get done without the eager execution in tensorflow.


# Tensorflow Operations Without Eager Execution Mode


So, there is a **disable_eager_execution()** function in TensorFlow 2.x. You can call the function like this:


In [3]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

#### Note: This function can only be called at the beginning before any Graphs, Ops, or Tensors have been  created.


Now, verify that the eager execution is disabled or not by running the below code.


In [4]:
tf.executing_eagerly()

False

As you can see **False** in the output that means it is disabled now.


Just execute the next cell. You will notice that we've created an object **a** of type **tensorflow.python.framework.ops.Tensor**.


In [5]:
import numpy as np
a = tf.constant(np.array([1., 2., 3.]))
type(a)

tensorflow.python.framework.ops.Tensor

Let's create another Tensor **b** and apply the dot product between them. This gives us **c**:


In [6]:
b = tf.constant(np.array([4.,5.,6.]))
c = tf.tensordot(a, b, 1)
type(c)

tensorflow.python.framework.ops.Tensor

In [7]:
print(c)

Tensor("Tensordot:0", shape=(), dtype=float64)


Note that **c** is a **tensorflow.python.framework.ops.Tensor** as well. So any node of the execution graph resembles a Tensor type. **But so far not a single computation happened**. You need to execute the graph. You can pass any graph or subgraph to the TensorFlow runtime for execution. Each TensorFlow graph runs within a TensorFlow Session, therefore we need to create it first:


**Note:** Session can be accessed via **tf.compat.v1.Session()** in Tensorflow 2.x.


In [8]:
session = tf.compat.v1.Session()
output = session.run(c)
session.close()
print(output)

32.0


2024-07-09 16:21:55.177392: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-09 16:21:55.179241: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Now you see the correct result of 32. But the problem is that debugging is pretty hard if you can only run complete graphs.


So let's actually re-enable the **eager execution**.


## Tensorflow Operations With Eager Execution Mode


### IMPORTANT! => Please don't forget restart the kernel by clicking on "Kernel" -> "Restart" so that the changes take effect.

**Enable or Disable Eager execution has to happen on program startup. This is the reason we have to restart the kernel.**

**After you have restarted the kernel, you may continue on to the next step.** 


Import the required libraries again.


In [1]:
import tensorflow as tf
import numpy as np

Run the below command to re-enable the eager execution.


In [2]:
from tensorflow.python.framework.ops import enable_eager_execution
enable_eager_execution()

Now you can run TensorFlow operations and the results will return immediately:


In [3]:
x = [[4]]
m = tf.matmul(x, x)
print("Result, {}".format(m))

Result, [[16]]


2024-07-09 16:22:37.773451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Enabling eager execution changes how TensorFlow operations behave—now they immediately evaluate and return their values to Python.


Since there isn't a computational graph to build and run later in a session, it's easy to inspect results using print() or a debugger.


In [4]:
a = tf.constant(np.array([1., 2., 3.]))
type(a)

tensorflow.python.framework.ops.EagerTensor

So the very same code created a different type of object. So now **a** is of type **tensorflow.python.framework.ops.EagerTensor**. This is great, because without changing code we obtain a tensor object which allows us to have a look inside, without execting a graph in a session:


In [5]:
print(a.numpy())

[1. 2. 3.]


Isn't this amazing? So from now on we can treat Tensors like ordinary python objects, work with them as usual, insert debug statements at any point or even use a debugger. So let's continue this example:


In [6]:
b = tf.constant(np.array([4.,5.,6.]))
c = tf.tensordot(a, b,1)
type(c)

tensorflow.python.framework.ops.EagerTensor

Again, **c** is an **tensorflow.python.framework.ops.EagerTensor** object which can be directly read:


In [7]:
print(c.numpy())

32.0


Without creating a session or a graph we obtained the result of the defined computation.


# Dynamic Control Flow

A major benefit of eager execution is that all the functionality of the host language is available while your model is executing. So, for example, it is easy to write [fizzbuzz](https://en.wikipedia.org/wiki/Fizz_buzz?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0120ENSkillsNetwork954-2023-01-01):


In [8]:
def fizzbuzz(max_num):
  counter = tf.constant(0)
  max_num = tf.convert_to_tensor(max_num)
  for num in range(1, max_num.numpy()+1):
    num = tf.constant(num)
    if int(num % 3) == 0 and int(num % 5) == 0:
      print('FizzBuzz')
    elif int(num % 3) == 0:
      print('Fizz')
    elif int(num % 5) == 0:
      print('Buzz')
    else:
      print(num.numpy())
    counter += 1

In [9]:
fizzbuzz(15)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


It prints these values at runtime. It behaves just like any other Python code. It is direct and intuitive. We can use pure Python if, while, and for in the control flow. So that's it for now, stay tuned and have fun.


<h1>Thank you for completing this notebook</h1>


## Author

<a href="https://www.linkedin.com/in/shubham-kumar-yadav-14378768?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0120ENSkillsNetwork954-2023-01-01">Shubham Yadav</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2020-09-04       | 1.0     | Lavanya    | Added lab to demonstrate Tensorflow eager execution |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
